In [119]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.models import load_model
from keras.layers import Input
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

In [120]:
movie_info = pd.read_csv('../data/interim/movie_rating_matrix.csv')
X_test = pd.read_csv('../data/interim/X_test_7.csv')

unseen = []
unseen.append(movie_info[f'rating'][1:])
for i in range(1, 1682):
#     print(movie_info[f'rating.{i}'][0])
    unseen.append(movie_info[f'rating.{i}'][1:])
          
ratings_test = X_test['ratings']

scaler = StandardScaler()
X = pd.read_csv('../data/raw/ml-100k/ub.test', sep='\t')
columns = ['user_id', 'item_id', 'ratings', 'timestamp']
X.columns = columns
rat = X['ratings']

scaler.fit(np.array(rat).reshape(-1, 1))

C:\Users\schir\AppData\Local\Temp\ipykernel_2016\3255645843.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_info = pd.read_csv('../data/interim/movie_rating_matrix.csv')


StandardScaler()

In [134]:
def top10(user_test, movie_test, model_mf, model_dnn_info):

    y_pred_mf = model_mf.predict([user_test, movie_test])
    y_pred_dnn_info = model_dnn_info.predict([user_test, movie_test])

    y_pred_mf = np.array(y_pred_mf).reshape(len(user_test))
    y_pred_dnn_info = np.array(y_pred_dnn_info).reshape(len(movie_test))
    
    y_pred_mf = scaler.inverse_transform(np.array(y_pred_mf).reshape(-1, 1))
    y_pred_dnn_info = scaler.inverse_transform(np.array(y_pred_dnn_info).reshape(-1, 1))

    user_recom_dnn = []
    res=[]
        
    for i in range(len(y_pred_dnn_info)):
        user_recom_dnn.append(y_pred_dnn_info[i])

    sorted_dnn = sorted(enumerate(user_recom_dnn), reverse=True, key=lambda i: i[1])
    for i in sorted_dnn[:10]:
        res.append(i[0])
    return res

In [135]:
# Evaluate model, trained on all 7 datasets(u1, u2, u3, u4, u5, ua, ub)
model_mf = load_model('../models/train_all/u7/mf_model_e01.hdf5')
model_dnn_info = load_model('../models/train_all/u7/dnn_info_model_e01.hdf5')
print("Metrics of the model, trained on all 7 datasets")
res=[]

for j in range(1, 944):
    user_test = []
    movie_test = []
    for i in range(1682):
        if unseen[i][j] == 0:
            user_test.append(j)
            movie_test.append(i+1)

    res.append(top10(np.array(user_test).reshape(-1, 1), np.array(movie_test).reshape(-1, 1), model_mf, model_dnn_info))

print(res[0])

Metrics of the model, trained on all 7 datasets
51/51 [==============================] - 0s 3ms/step


51/51 [==============================] - 0s 3ms/step


52/52 [==============================] - 0s 3ms/step


41/41 [==============================] - 0s 3ms/step


44/44 [==============================] - 0s 3ms/step


...

48/48 [==============================] - 0s 3ms/step


51/51 [==============================] - 0s 3ms/step


48/48 [==============================] - 0s 3ms/step


50/50 [==============================] - 0s 4ms/step


48/48 [==============================] - 0s 4ms/step
[849, 902, 1176, 1194, 1383, 1175, 1227, 1020, 1356, 541]


In [136]:
print("10 best for user with id 1: ", res[0])

10 best for user with id 1:  [849, 902, 1176, 1194, 1383, 1175, 1227, 1020, 1356, 541]
